In [1]:
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Dense, Flatten, Input


visible = Input(shape=(120, 320, 3))
conv1 = Conv2D(16, (3,3), activation='relu')(visible)
pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
conv2 = Conv2D(32, (3,3), activation='relu')(pool1)
pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
conv3 = Conv2D(32, (3,3), activation='relu')(pool2)
flat = Flatten()(conv3)
hidden = Dense(32, activation='relu')(flat)

output1 = Dense(11, activation = 'softmax')(hidden)
output2 = Dense(11, activation = 'softmax')(hidden)
output3 = Dense(11, activation = 'softmax')(hidden)
output4 = Dense(11, activation = 'softmax')(hidden)

model = Model(inputs = visible, outputs = [output1, output2, output3, output4])

model.compile(optimizer='adam', loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy','categorical_crossentropy'], metrics=['accuracy'])
model.summary()


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 120, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 118, 318, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 59, 159, 16)  0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 57, 157, 32)  4640        max_pooling2d_1[0][0]            
_____________________________________

In [2]:
training_img_paths = []
validation_img_paths = []


for i in range(4500, 5000):
    training_img_paths += ['digits data/boards/training/board_'+str(i)+'.jpg']
    
for i in range(500):
    validation_img_paths += ['digits data/boards/validation/board_'+str(i)+'.jpg']

In [3]:
import cv2 
import numpy as np

def get_input(path):
    img = cv2.imread(path)
    return img 

In [4]:
def get_output(path):
    output1 = np.zeros(11)
    output2 = np.zeros(11)
    output3 = np.zeros(11)
    output4 = np.zeros(11)
    
    img_id = path.split('_')[-1].split('.')[0]
    label = np.load('digits data/labels/label_'+img_id+'.npy')
    
    output1[label[0]] = 1
    output2[label[1]] = 1
    output3[label[2]] = 1
    output4[label[3]] = 1
    
    return output1, output2, output3, output4

In [6]:
def image_generator(files, batch_size):
    
    while True:
        # Select files (paths/indices) for the batch
        batch_paths = np.random.choice(a = files, 
                                         size = batch_size)
        batch_input = []
        batch_output1 = [] 
        batch_output2 = []
        batch_output3 = []
        batch_output4 = []
          
        # Read in each input, perform preprocessing and get labels
        for input_path in batch_paths:
            input = get_input(input_path)
            batch_output = get_output(input_path)

            #input = preprocess_input(image=input) 
            batch_input += [ input ]
            batch_output1 += [ batch_output[0] ]
            batch_output2 += [ batch_output[1] ]
            batch_output3 += [ batch_output[2] ]
            batch_output4 += [ batch_output[3] ]
            
        # Return a tuple of (input,output) to feed the network
        batch_x = np.array( batch_input )
        batch_y1 = np.array( batch_output1 )
        batch_y2 = np.array( batch_output2 )
        batch_y3 = np.array( batch_output3 )
        batch_y4 = np.array( batch_output4 )
        
        yield( batch_x, [batch_y1, batch_y2, batch_y3, batch_y4])

In [7]:
batch_size = 32

training_gen = image_generator(training_img_paths, batch_size)
validation_gen = image_generator(validation_img_paths, batch_size)

In [8]:
model.fit_generator(training_gen, steps_per_epoch=125, epochs=5, validation_data=validation_gen, 
                   validation_steps=20)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
125/125 [==============================] - 124s 994ms/step - loss: 58.1819 - dense_2_loss: 14.2789 - dense_3_loss: 14.5552 - dense_4_loss: 14.5749 - dense_5_loss: 14.7728 - dense_2_acc: 0.1135 - dense_3_acc: 0.0963 - dense_4_acc: 0.0953 - dense_5_acc: 0.0830 - val_loss: 58.9570 - val_dense_2_loss: 14.6574 - val_dense_3_loss: 14.5315 - val_dense_4_loss: 14.6070 - val_dense_5_loss: 15.1611 - val_dense_2_acc: 0.0906 - val_dense_3_acc: 0.0984 - val_dense_4_acc: 0.0938 - val_dense_5_acc: 0.0594
Epoch 2/5
125/125 [==============================] - 121s 966ms/step - loss: 57.9848 - dense_2_loss: 14.2323 - dense_3_loss: 14.5264 - dense_4_loss: 14.4902 - dense_5_loss: 14.7360 - dense_2_acc: 0.1170 - dense_3_acc: 0.0988 - dense_4_acc: 0.1010 - dense_5_acc: 0.0858 - val_loss: 59.1584 - val_dense_2_loss: 14.7078 - val_dense_3_loss: 14.8841 - val_dense_4_loss: 14.5567 - val_dense_5_loss: 15.0100 - val_dense_2_acc: 0.0875 - val_dense_3_acc: 0

In [101]:
import h5py 
model.save_weights('first_try_digits.h5')

In [ ]:
import cv2 
import numpy as np

img = cv2.imread('digits data/test.png')
print(img.shape)
img = cv2.resize(img, (320, 120))
img = img_to_array(img)

plt


tab = np.zeros((2, 120, 320, 3))
tab[0] = img 
tab[1] = img 

model.predict(tab)

In [9]:
import numpy as np 
from random import seed
np.random.seed(97)
indices = np.random.randint(0, 5000, 32)
np.random.seed(97)
indices2 = np.random.randint(0, 5000, 32)
print(indices)
print(indices2)

[2467  105 2997 3271 3356 2227 4933 2308 4600  850 1907 1309 4948 2564
  419 3984 2236 1496 2986 1219 1527 1625 2238 4601 2883 3917 4396 4332
   39   13 2511  863]
[2467  105 2997 3271 3356 2227 4933 2308 4600  850 1907 1309 4948 2564
  419 3984 2236 1496 2986 1219 1527 1625 2238 4601 2883 3917 4396 4332
   39   13 2511  863]


In [43]:
bail = get_output('digits data/boards/board_2176.jpg')
print(bail)
print(bail[0])

(array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]), array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]))
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [26]:
txt = 'digits data/boards/board_2176.jpg'

img_id = txt.split('_')[-1].split('.')[0]
print(img_id)



2176
